In [ ]:
import numpy as np
import pandas as pd
import tensorflow as tf
from keras.models import Model
from keras.layers import Activation, AvgPool1D, Dense, Conv1D, Flatten, Dropout, Input, BatchNormalization, GlobalMaxPool1D, MaxPool1D, SpatialDropout1D, GlobalAvgPool1D
from keras.optimizers import Adam
#from keras.utils import np_utils
from tensorflow.keras.utils import to_categorical

from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import confusion_matrix
from scipy import signal
import pickle as pkl

In [ ]:
import numpy as np
import pandas as pd
from scipy import signal
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.utils import to_categorical  # Sử dụng thư viện phù hợp để chuyển đổi nhãn sang one-hot
selected_channels = [channel_indices['AF3'], channel_indices['AF4'], channel_indices['T7'], channel_indices['T8']]
def load_data(eeg_band=None):
    data = []
    val_label = []
    aro_label = []
    dom_label = []
    base_path = '/content/drive/MyDrive/DREAMER DATA/Process_Data'  # Đường dẫn cơ sở cho dữ liệu

    for person in range(1, 24):
        print('Person No. ' + str(person))
        person_path = f"{base_path}/person{person}"

        # Tải các nhãn
        valence = pd.read_csv(f"{person_path}/eeg_labels/valence.csv", header=None).values.ravel()
        arousal = pd.read_csv(f"{person_path}/eeg_labels/arousal.csv", header=None).values.ravel()
        dominance = pd.read_csv(f"{person_path}/eeg_labels/dominance.csv", header=None).values.ravel()

        # Chuyển đổi nhãn sang lớp phân loại dựa trên ngưỡng
        valence = (valence > 3).astype(int)
        arousal = (arousal > 3).astype(int)
        dominance = (dominance > 3).astype(int)

        # Xử lý và phân đoạn dữ liệu EEG
        for i in range(1, 19):
            eeg = pd.read_csv(f"{person_path}/eeg_samples/eeg{i}.csv", header=None).values
            # Chỉ lấy dữ liệu từ các kênh đã chọn
            eeg = eeg[:, selected_channels]
            num, den = signal.butter(4, bands[eeg_band], 'band')  # Tạo bộ lọc Butterworth
            eeg = signal.filtfilt(num, den, eeg, axis=0)

            scaler = StandardScaler().fit(eeg)
            eeg = scaler.transform(eeg)

            # Phân đoạn dữ liệu
            start = 0
            while start + 1280 < eeg.shape[0]:
                end = start + 1280
                data.append(eeg[start:end])
                val_label.append(valence[i-1])  # Nhãn cho mỗi phân đoạn
                aro_label.append(arousal[i-1])
                dom_label.append(dominance[i-1])
                start += 256  # Độ chồng lấp

    data = np.array(data, dtype=np.float32)
    val_label = to_categorical(np.array(val_label, dtype=np.int8))
    aro_label = to_categorical(np.array(aro_label, dtype=np.int8))
    dom_label = to_categorical(np.array(dom_label, dtype=np.int8))

    return data, val_label, aro_label, dom_label

# Các biến để định nghĩa băng tần
bands = {
    'delta': [0.5/64, 4/64],
    'theta': [4/64, 8/64],
    'alpha': [8/64, 14/64],
    'beta': [14/64, 30/64],
    'gamma': [30/64, 50/64]
}


In [ ]:
import numpy as np

FOLD = 10  # Number of folds
eeg_band = 'theta'  # EEG band name

# Assuming load_data is imported and correctly implemented as discussed
data, valence, arousal, dominance = load_data(eeg_band)  # Loading processed data

nb_samples = data.shape[0]  # Number of samples
factor = nb_samples // FOLD  # Kth fold by this factor

# Shuffling data
shuffler = np.random.permutation(nb_samples)
data = data[shuffler]
valence = valence[shuffler]
arousal = arousal[shuffler]
dominance = dominance[shuffler]

# Now data and labels are shuffled and ready to be split into folds.


Person No. 1
Person No. 2
Person No. 3
Person No. 4
Person No. 5
Person No. 6
Person No. 7
Person No. 8
Person No. 9
Person No. 10
Person No. 11
Person No. 12
Person No. 13
Person No. 14
Person No. 15
Person No. 16
Person No. 17
Person No. 18
Person No. 19
Person No. 20
Person No. 21
Person No. 22
Person No. 23


In [ ]:
from tensorflow.keras.layers import Input, Conv1D, AvgPool1D, BatchNormalization, SpatialDropout1D, GlobalAvgPool1D, Dropout, Dense, Activation
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam

# Định nghĩa Input ở bên ngoài hàm get_model
eeg_input = Input(shape=(window_size, 4), name='eeg_input')

def get_CNN(eeg_input):
    x = Conv1D(filters=32, kernel_size=5, strides=2, padding='valid', activation='relu', name='conv1')(eeg_input)
    # Thêm các layer tiếp theo như trước
    x = Conv1D(filters=512, kernel_size=3, strides=1, padding='valid', activation='relu', name='conv8')(x)
    x = GlobalAvgPool1D(name='global_pool1')(x)
    x = BatchNormalization(name='batch_norm4')(x)
    x = Dropout(0.5)(x)
    x = Dense(64)(x)
    x = Activation('tanh')(x)
    x = Dense(8)(x)
    x = Activation('tanh')(x)
    x = Dropout(0.25)(x)
    return x

def get_model(eeg_input):
    x = get_CNN(eeg_input)
    out = Dense(classes, activation='softmax', name='output')(x)  # Output layer
    model = Model(inputs=eeg_input, outputs=out)  # Tạo instance của Model
    adam = Adam(learning_rate=1e-3)  # Adam optimizer
    model.compile(optimizer=adam, loss='categorical_crossentropy', metrics=['categorical_accuracy'])  # Biên dịch model
    model.summary()
    return model


In [ ]:
import numpy as np

In [ ]:
import numpy as np
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input
from sklearn.metrics import confusion_matrix
import pickle as pkl
from sklearn.model_selection import KFold

# Khởi tạo biến kết quả
val_res = {'accuracy': [], 'confusion_matrix': []}
FOLD = 10
nb_samples = data.shape[0]
kfold = KFold(n_splits=FOLD, shuffle=True, random_state=42)  # K-fold Cross-Validation

for train_index, test_index in kfold.split(data):
    X_train, X_test = data[train_index], data[test_index]
    val_train, val_test = valence[train_index], valence[test_index]

    model = get_model(eeg_input)  # Tạo mô hình mới
    history = model.fit(X_train, val_train, epochs=100, batch_size=256, shuffle=True)

    # In kết quả huấn luyện
    print("Training Loss:", history.history['loss'][-1])
    print("Training Accuracy:", history.history['categorical_accuracy'][-1])

    # Đánh giá mô hình trên tập kiểm tra
    loss, accuracy = model.evaluate(X_test, val_test)
    print("Test Loss:", loss)
    print("Test Accuracy:", accuracy)

    # Dự đoán và tính toán confusion matrix
    pred = model.predict(X_test)
    cm = confusion_matrix(val_test.argmax(axis=1), pred.argmax(axis=1))
    val_res['accuracy'].append(accuracy)
    val_res['confusion_matrix'].append(cm)
    print("Confusion Matrix:\n", cm)

# Lưu kết quả vào file
file_path = '/content/drive/MyDrive/DREAMER DATA/Process_Data/Dreamer_valence_' + str(eeg_band) + '.pkl'
with open(file_path, 'wb') as f:
    pkl.dump(val_res, f)



Model: "model_9"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 eeg_input (InputLayer)      [(None, 1280, 4)]         0         
                                                                 
 conv1 (Conv1D)              (None, 638, 32)           672       
                                                                 
 conv8 (Conv1D)              (None, 636, 512)          49664     
                                                                 
 global_pool1 (GlobalAverag  (None, 512)               0         
 ePooling1D)                                                     
                                                                 
 batch_norm4 (BatchNormaliz  (None, 512)               2048      
 ation)                                                          
                                                                 
 dropout_9 (Dropout)         (None, 512)               0   

KeyboardInterrupt: 

In [ ]:
print(history.history)


{'loss': [0.6796363592147827, 0.6256839036941528, 0.5998565554618835, 0.5802474021911621, 0.5617167353630066, 0.5385253429412842, 0.5163530707359314, 0.49033159017562866, 0.46594810485839844, 0.44155779480934143, 0.4175613224506378, 0.3976639211177826, 0.37849175930023193, 0.3594135642051697, 0.34315407276153564, 0.32249388098716736, 0.31331783533096313, 0.29782161116600037, 0.2848975360393524, 0.27292975783348083, 0.2632759213447571, 0.2553407847881317, 0.2409684658050537, 0.2328132688999176, 0.22740982472896576, 0.21585184335708618, 0.21196343004703522, 0.20380455255508423, 0.19598229229450226, 0.18937943875789642, 0.18226902186870575, 0.17491918802261353, 0.17238061130046844, 0.16943109035491943, 0.16574928164482117, 0.16032801568508148, 0.1491883248090744, 0.14827701449394226, 0.14574892818927765, 0.14172005653381348, 0.14209069311618805, 0.13243931531906128, 0.13433803617954254, 0.12666553258895874, 0.12903709709644318, 0.1278049796819687, 0.11795727163553238, 0.11393167078495026,

In [ ]:
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Conv1D, BatchNormalization, Activation, AveragePooling1D, Flatten, Dense, Dropout, GlobalAveragePooling1D, Reshape, Multiply
from tensorflow.keras.optimizers import Adam

# Squeeze-Excite block
def squeeze_excite_block(input, ratio=16):
    init = input
    channel_axis = -1
    filters = init.shape[channel_axis]
    se_shape = (1, filters)

    se = GlobalAveragePooling1D()(init)
    se = Reshape(se_shape)(se)
    se = Dense(filters // ratio, activation='relu', kernel_initializer='he_normal', use_bias=False)(se)
    se = Dense(filters, activation='sigmoid', kernel_initializer='he_normal', use_bias=False)(se)

    x = Multiply()([init, se])
    return x

# MCSmini architecture
def get_MCSmini(input_shape, num_classes):
    input_layer = Input(shape=input_shape)

    x = Conv1D(32, kernel_size=3, padding='same', activation='relu')(input_layer)
    x = BatchNormalization()(x)
    x = squeeze_excite_block(x)  # Add SE block after the first Conv1D layer
    x = AveragePooling1D(pool_size=2)(x)

    x = Conv1D(64, kernel_size=3, padding='same', activation='relu')(x)
    x = BatchNormalization()(x)
    x = squeeze_excite_block(x)  # Add SE block
    x = AveragePooling1D(pool_size=2)(x)

    x = Flatten()(x)
    x = Dense(128, activation='relu')(x)
    x = Dropout(0.5)(x)  # Increase dropout to reduce overfitting

    output_layer = Dense(num_classes, activation='softmax')(x)

    model = Model(inputs=input_layer, outputs=output_layer)
    model.compile(optimizer=Adam(learning_rate=1e-3), loss='categorical_crossentropy', metrics=['categorical_accuracy'])
    model.summary()  # To see the structure of your network
    return model

# Use the defined architecture to create the model
eeg_input_shape = (window_size, 4)  # Adjusted to the correct number of channels
num_classes = 2  # Assuming binary classification
model = get_MCSmini(eeg_input_shape, num_classes)


Model: "model_6"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_7 (InputLayer)        [(None, 1280, 4)]            0         []                            
                                                                                                  
 conv1d_12 (Conv1D)          (None, 1280, 32)             416       ['input_7[0][0]']             
                                                                                                  
 batch_normalization_12 (Ba  (None, 1280, 32)             128       ['conv1d_12[0][0]']           
 tchNormalization)                                                                                
                                                                                                  
 global_average_pooling1d_1  (None, 32)                   0         ['batch_normalization_12

In [ ]:
import numpy as np
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input
from sklearn.metrics import confusion_matrix
import pickle as pkl
from sklearn.model_selection import KFold

# Định nghĩa lại hàm get_model() để sử dụng get_MCSmini
def get_model(input_shape, num_classes):
    return get_MCSmini(input_shape, num_classes)

# Khởi tạo biến kết quả
val_res = {'accuracy': [], 'confusion_matrix': []}
FOLD = 5
nb_samples = data.shape[0]
kfold = KFold(n_splits=FOLD, shuffle=True, random_state=42)  # K-fold Cross-Validation

# Kích thước đầu vào dựa trên cấu hình của dữ liệu và số kênh đã chọn
input_shape = (window_size, 4)  # 5 kênh như đã thảo luận

# Huấn luyện mô hình trong mỗi fold
for train_index, test_index in kfold.split(data):
    X_train, X_test = data[train_index], data[test_index]
    val_train, val_test = valence[train_index], valence[test_index]

    # Tạo mô hình mới cho mỗi fold để tránh leak thông tin giữa các fold
    model = get_model(input_shape, 2)  # Giả sử là phân loại nhị phân
    history = model.fit(X_train, val_train, epochs=100, batch_size=256, shuffle=True)

    # In kết quả huấn luyện
    print("Training Loss:", history.history['loss'][-1])
    print("Training Accuracy:", history.history['categorical_accuracy'][-1])

    # Đánh giá mô hình trên tập kiểm tra
    loss, accuracy = model.evaluate(X_test, val_test)
    print("Test Loss:", loss)
    print("Test Accuracy:", accuracy)

    # Dự đoán và tính toán confusion matrix
    pred = model.predict(X_test)
    cm = confusion_matrix(val_test.argmax(axis=1), pred.argmax(axis=1))
    val_res['accuracy'].append(accuracy)
    val_res['confusion_matrix'].append(cm)
    print("Confusion Matrix:\n", cm)

# Lưu kết quả vào file
file_path = '/content/drive/MyDrive/DREAMER DATA/Process_Data/Dreamer_valenceMCSmini_' + str(eeg_band) + '.pkl'
with open(file_path, 'wb') as f:
    pkl.dump(val_res, f)


Model: "model_7"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_8 (InputLayer)        [(None, 1280, 4)]            0         []                            
                                                                                                  
 conv1d_14 (Conv1D)          (None, 1280, 32)             416       ['input_8[0][0]']             
                                                                                                  
 batch_normalization_14 (Ba  (None, 1280, 32)             128       ['conv1d_14[0][0]']           
 tchNormalization)                                                                                
                                                                                                  
 global_average_pooling1d_1  (None, 32)                   0         ['batch_normalization_14

KeyboardInterrupt: 